<a href="https://colab.research.google.com/github/Asrithareddy19/hate_speech_detection/blob/main/Hate_speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

model


In [ ]:
#defines a neural network model for hate speech detection using the BERT
import transformers
from transformers import AutoTokenizer
from transformers import BertForTokenClassification, BertForSequenceClassification,BertPreTrainedModel, BertModel
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
import numpy as np

class BertPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output
class Model_Rational_Label(BertPreTrainedModel):
     def __init__(self,config):
        super().__init__(config)
        #### Keep this parameters fixed
        self.num_labels=2
        self.impact_factor=10
        ####
        self.bert = BertModel(config,add_pooling_layer=False)
        self.bert_pooler=BertPooler(config)
        self.token_dropout = nn.Dropout(0.1)
        self.token_classifier = nn.Linear(config.hidden_size, 2)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(config.hidden_size, self.num_labels)
        self.init_weights()

     def forward(self, input_ids=None, mask=None, attn=None, labels=None):
        outputs = self.bert(input_ids, mask)
        out=outputs[0]
        logits = self.token_classifier(self.token_dropout(out))
        embed=self.bert_pooler(outputs[0])
        y_pred = self.classifier(self.dropout(embed))
        loss_token = None
        loss_label = None
        loss_total = None

        if attn is not None:
            loss_fct = nn.CrossEntropyLoss()
            # Only keep active parts of the loss
            if mask is not None:
                active_loss = mask.view(-1) == 1
                active_logits = logits.view(-1, 2)
                active_labels = torch.where(
                    active_loss, attn.view(-1), torch.tensor(loss_fct.ignore_index).type_as(attn)
                )
                loss_token = loss_fct(active_logits, active_labels)
            else:
                loss_token = loss_fct(logits.view(-1, 2), attn.view(-1))

            loss_total=self.impact_factor*loss_token


        if labels is not None:
            loss_funct = nn.CrossEntropyLoss()
            loss_logits =  loss_funct(y_pred.view(-1, self.num_labels), labels.view(-1))
            loss_label= loss_logits
            if(loss_total is not None):
                loss_total+=loss_label
            else:
                loss_total=loss_label
        if(loss_total is not None):
            return y_pred, logits, loss_total
        else:
            return y_pred, logits

utils

In [ ]:
import numpy as np
import json
def softmax(x):    #calculates probabilities
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    temp=e_x / e_x.sum(axis=0) # only difference

    if np.isnan(temp).any()==True:
        return [0.0,1.0,0.0]
    else:
        return temp
class CharVal(object):
    def __init__(self, char, val):
        self.char = char
        self.val = val

    def __str__(self):
        return self.char

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb


def color_charvals_lime(s):
    r = 255-int(s.val*255)
    color = rgb_to_hex((255, r, r))
    return 'background-color: %s' % color
def color_charvals_rationale(s):
    r = 255-int(s.val*255)
    color = rgb_to_hex((255, r, r))
    return 'background-color: %s' % color


predictions

In [ ]:
pip install ekphrasis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.8 MB/s eta 0:00:00


In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import re
from transformers import AutoTokenizer,AutoModelForSequenceClassification,AutoConfig
import numpy as np
import torch
#fixing up typos, removing website links, also identifies and tags things like hashtags and repeated words
text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    fix_html=True,  # fix HTML tokens
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    segmenter="twitter",
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons]
)


# this class encapsulates the functionality to preprocess input text, tokenize it,
#perform model inference, and extract attention vectors, providing a streamlined approach to predict rational labels for a given set of sentence
class modelPredRationale():
    def __init__(self, model_path = 'bert-base-uncased', device = None):
        self.device = device
        self.model_path=model_path
        self.model = Model_Rational_Label.from_pretrained(model_path,output_attentions = True,output_hidden_states = False).to(self.device)
        self.config = AutoConfig.from_pretrained(self.model_path)
        if torch.cuda.is_available():
            self.model.cuda()
        self.model.eval()
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path, use_fast = False)

    def preprocess_func(self, text):
        remove_words=['<allcaps>','</allcaps>','<hashtag>','</hashtag>','<elongated>','<emphasis>','<repeated>','\'','s']
        word_list=text_processor.pre_process_doc(text)
        word_list=list(filter(lambda a: a not in remove_words, word_list))
        sent=" ".join(word_list)
        sent = re.sub(r"[<\*>]", " ",sent)
        return sent

    def tokenize(self, sentences, padding = True, max_len = 128):
        input_ids, attention_masks, token_type_ids = [], [], []
        for sent in sentences:
            encoded_dict = self.tokenizer.encode_plus(sent,
                                                    add_special_tokens=True,
                                                    max_length=max_len,
                                                    padding='max_length',
                                                    return_attention_mask = True,
                                                    return_tensors = 'pt',
                                                    truncation = True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return {'input_ids': input_ids, 'attention_masks': attention_masks}

    def process_data(self, sentences_list):
        sentences = []
        sentence_lengths = []
        for sentence in sentences_list:
            try:
                sentence = self.preprocess_func(sentence)
            except TypeError:
                sentence = self.preprocess_func("dummy text")
            sentences.append(sentence)
            sentence_lengths.append(len(self.tokenizer.encode(sentence)))
        inputs = self.tokenize(sentences)
        tokenized_sentences = [self.tokenizer.convert_ids_to_tokens(ele) for ele in  inputs['input_ids']]

        return self.get_dataloader(inputs), sentence_lengths, tokenized_sentences

    def get_dataloader(self, inputs):
        data = TensorDataset(inputs['input_ids'], inputs['attention_masks'])
        sampler = SequentialSampler(data)
        return DataLoader(data, sampler=sampler, batch_size=32)


    def return_rationales(self, sentences_list):
        """Input: should be a list of sentences"""
        """Output: probablity values"""
        device = self.device

        test_dataloader,sentence_lengths, tokenized_sentences=self.process_data(sentences_list)

        print("Running eval on test data...")
        labels_list=[]
        rationale_list=[]
        rationale_logit_list = []
        # Evaluate data
        for step,batch in enumerate(test_dataloader):

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)

            label_logits, rationale_logits = self.model(b_input_ids, b_input_mask)

            label_logits = label_logits.detach().cpu().numpy()
            rationale_logits = rationale_logits.detach().cpu().numpy()

            final_logits=[]
            final_rationales=[]
            for i in range(label_logits.shape[0]):
                final_logits.append(softmax(label_logits[i]))
                final_rationales.append([ele[1] for ele in rationale_logits[i]])
            labels_list+=final_logits
            rationale_list+=final_rationales

        attention_vectors = []
        for idx, rationales in enumerate(rationale_list):
            attention_vector = softmax(rationales[:sentence_lengths[idx]])
            attention_vector = list(attention_vector) + [0]*(128-len(list(attention_vector)))
            attention_vectors.append(attention_vector)

        tokens_sentence=[]
        for idx, tokenized in enumerate(tokenized_sentences):
            tokenized = tokenized[:sentence_lengths[idx]]
            tokens_sentence.append(tokenized)

        return np.array(labels_list), np.array(attention_vectors), tokens_sentence


#this class encapsulates the functionality required to preprocess text,
#tokenize it, pass it through a pre-trained language model, and obtain predictions for text classification tasks.
class modelPred():
    def __init__(self, language='english', device=None):
        self.__modelDict ={
        'arabic':"Hate-speech-CNERG/dehatebert-mono-arabic",
        'english': "Hate-speech-CNERG/dehatebert-mono-english",
        'english_hatexplain':"Hate-speech-CNERG/bert-base-uncased-hatexplain",
        'french': "Hate-speech-CNERG/dehatebert-mono-french",
        'german': "Hate-speech-CNERG/dehatebert-mono-german",
        'indonesian': "Hate-speech-CNERG/dehatebert-mono-indonesian",
        'polish': "Hate-speech-CNERG/dehatebert-mono-polish",
        'portugese': "Hate-speech-CNERG/dehatebert-mono-portugese",
        'italian': "Hate-speech-CNERG/dehatebert-mono-italian",
        'spanish': "Hate-speech-CNERG/dehatebert-mono-spanish"
        }
        self.device = device
        self.model_path=self.__modelDict[language]
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
        self.config = AutoConfig.from_pretrained(self.model_path)
        # if(model_name=='xlmr'):
        #     self.model = XLMRobertaForSequenceClassification.from_pretrained(self.model_path,output_attentions = True,output_hidden_states = False).to(self.device)
        # elif(model_name=='bert'):
        #     self.model = BertForSequenceClassification.from_pretrained(self.model_path,output_attentions = True,output_hidden_states = False).to(self.device)
        self.model.cuda()
        self.model.eval()

    def preprocess_func(self, text):
        new_text = re.sub('@\w+', '@user',text)
        new_text = new_text.replace("\r\n\'",' ').replace("\n",' ')
        new_text = re.sub(r"http\S+", "", new_text)
        new_text = new_text.replace('&amp;', '&')
        return new_text

    def tokenize(self, sentences, padding = True, max_len = 128):
        input_ids, attention_masks, token_type_ids = [], [], []
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        for sent in sentences:
            encoded_dict = self.tokenizer.encode_plus(sent,
                                                    add_special_tokens=True,
                                                    max_length=max_len,
                                                    padding='max_length',
                                                    return_attention_mask = True,
                                                    return_tensors = 'pt',
                                                    truncation = True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return {'input_ids': input_ids, 'attention_masks': attention_masks}

    def process_data(self, sentences_list):
        sentences = []
        for sentence in sentences_list:
            try:
                sentence = self.preprocess_func(sentence)
            except TypeError:
                sentence = self.preprocess_func("dummy text")
            sentences.append(sentence)
        inputs = self.tokenize(sentences)
        return self.get_dataloader(inputs)

    def get_dataloader(self, inputs):
        data = TensorDataset(inputs['input_ids'], inputs['attention_masks'])
        sampler = SequentialSampler(data)
        return DataLoader(data, sampler=sampler, batch_size=32)

    def return_probab(self, sentences_list):
        """Input: should be a list of sentences"""
        """Output: probablity values"""
        device = self.device

        test_dataloader=self.process_data(sentences_list)

        print("Running eval on test data...")
        labels_list=[]
        sentence_lengths = [len(self.tokenizer.encode(sentence)) for sentence in  sentences_list]
        # Evaluate data
        for step,batch in enumerate(test_dataloader):

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)

            label_logits = self.model(b_input_ids, b_input_mask).logits
            label_logits = label_logits.detach().cpu().numpy()

            final_logits=[]
            for i in range(label_logits.shape[0]):
                final_logits.append(softmax(label_logits[i]))
            labels_list+=final_logits

        return np.array(labels_list)

# modelPredRationale is tailored for models providing explanations,
#while modelPred is a more general-purpose class for making predictions with language models.

/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


In [ ]:
%%capture
import transformers
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import re
import torch.nn.functional as F
import numpy as np

In [ ]:
if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")

In [ ]:
def getDatasetPrediction(dataset,config):
    labels=model.return_probab(dataset['Sentences'])
    predictions = {}
    for index, row in dataset.iterrows():

        dict1={}
        dict1['Sentence']=row['Sentences']
        dict_labels={}
        for ele in config:
            dict_labels[config[ele]]=round(labels[index][ele],3)
        dict1["Labels"]=dict_labels
        predictions[row['Index']] = dict1
    return predictions

def getRandomTextFromPred(pred = None):
    return random.choice(list(prediction.items()))

In [ ]:
#@title ### **Select a language**
Language = "Arabic" #@param ["Arabic", "English", "French", "German", "Indonesian", "Polish", "Portugese", "Italian", "Spanish", "English_hatexplain"]


In [ ]:
model = modelPred(language=Language.lower(), device=device)

In [ ]:
#@title **How do you want to enter text ?**
# @markdown You can either directly enter the text (text input) or uppload from a csv (file)
input_type = "text input" #@param ["file", "text input"]

In [ ]:
import io
import pandas as pd

if input_type == "text input":
    text_input = input("Write the post: ")
    dataset=[text_input]
else:
  print("Please upload the csv file you want to get predictions")
  print("Please make sure the column name of the csv should be Index, Sentences")
  from google.colab import files
  uploaded = files.upload()
  dataset = pd.read_csv(io.BytesIO(uploaded[list(uploaded)[0]])).reset_index()

Write the post:  قطر مرادف الارهاب يا حمار قمة يعني زبالة طيب كلي هوا وخليكي بزباله لبنان حسن عليق؟؟؟ كول هوا ابوك الحرامي طبعا يا عيلة وسخة بلا شرف ولا اخلاق


In [ ]:
if input_type == "text input":
  labels=model.return_probab(dataset)
  dict1={}
  dict1['Sentence']=dataset[0]
  dict_labels={}
  config=model.config.id2label
  for ele in config:
      dict_labels[config[ele]]=round(labels[0][ele],3)
  dict1["Labels"]=dict_labels
  print(dict1)
else:
  prediction = getDatasetPrediction(dataset,model.config.id2label)
  print(getRandomTextFromPred(prediction))


Running eval on test data...
{'Sentence': ' قطر مرادف الارهاب يا حمار قمة يعني زبالة طيب كلي هوا وخليكي بزباله لبنان حسن عليق؟؟؟ كول هوا ابوك الحرامي طبعا يا عيلة وسخة بلا شرف ولا اخلاق', 'Labels': {'NON_HATE': 0.025, 'HATE': 0.975}}


In [ ]:
pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 9.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=ef8f568d9185ba5d7465927ae250c89c7471585615bd7292dd636e075815d87f
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling 

In [ ]:
from googletrans import Translator

def translate(text, source_lang='auto', dest_lang='en'):
    translator = Translator()
    translated_text = translator.translate(text, src=source_lang, dest=dest_lang)
    return translated_text.text

text_to_translate = text_input

translated_text = translate(text_to_translate, dest_lang='en')
print("Translated text:", translated_text)


Translated text: Qatar is the synonym of terrorism, O donkey, a summit, I mean, the garbage is good, Hawa, and I will leave you, Lebanon, Hassan Alik ???Cole Hua, your father, the thief, of course, a dirty family, without honor or morals
